## TPC-DS BQ Query Validation

In [21]:
from google.cloud import bigquery

In [1]:
import ds_setup, config, tools

In [2]:
from importlib import reload

In [173]:
client = bigquery.Client.from_service_account_json(config.gcp_cred_file)
job_config = bigquery.QueryJobConfig()
job_config.default_dataset = "tpc-benchmarking-9432.ds_2GB_1_basic"

## =====

In [175]:
# this should have been done elsewhere
# ds_setup.tpl_bq_regex(tpl_dir=input_templates_bq, verbose=False)

In [176]:
test_name = "ds_01_1GB_bq_default_dev"

input_templates_bq = config.fp_ds_bq_template_dir

dialect = ds_setup.sqlserver_bq_defines(input_templates_bq)

In [168]:
def ds_qgen_tpl_number(n, directory, verbose=False):
    """Generate DS query text for query number n
    
    Parameters
    ----------|
    n : int, query number to generate BigQuery SQL
    directory : str, absolute path to directory of query templates
        to draw from for n.
        
    Returns
    -------
    str : BigQuery SQL query
    """
    std_out, err_out = ds_setup.dsqgen(directory=directory,
                                       dialect="sqlserver_bq",
                                       scale=1,
                                       template="query{}.tpl".format(n),
                                       filter="Y",
                                       qualify="Y")
    return std_out

In [244]:
n = 99

query_text = ds_qgen_tpl_number(n, config.fp_ds_bq_template_dir)

query_job = client.query(query_text, job_config=job_config)

result = query_job.result()

print("QUERY {} : query{}.tpl".format(n,n))
print(query_text)
print("-"*50)
print("QUERY {} : query{}.tpl".format(n,n))
print("="*20, "RESULTS", "="*20)
list(result)

QUERY 99 : query99.tpl

select  
   substr(w_warehouse_name,1,20)
  ,sm_type
  ,cc_name
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk <= 30 ) then 1 else 0 end)  as _30_days 
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 30) and 
                 (cs_ship_date_sk - cs_sold_date_sk <= 60) then 1 else 0 end )  as _31_60_days 
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 60) and 
                 (cs_ship_date_sk - cs_sold_date_sk <= 90) then 1 else 0 end)  as _61_90_days 
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk > 90) and
                 (cs_ship_date_sk - cs_sold_date_sk <= 120) then 1 else 0 end)  as _91_120_days 
  ,sum(case when (cs_ship_date_sk - cs_sold_date_sk  > 120) then 1 else 0 end)  as _greater_than_120_days 
from
   catalog_sales
  ,warehouse
  ,ship_mode
  ,call_center
  ,date_dim
where
    d_month_seq between 1212 and 1212 + 11
and cs_ship_date_sk   = d_date_sk
and cs_warehouse_sk   = w_warehouse_sk
and cs_ship_mode_sk   = sm_ship_mode_sk

[Row((None, 'EXPRESS', 'California', 3006, 3136, 3136, 0, 0), {'f0_': 0, 'sm_type': 1, 'cc_name': 2, '_30_days': 3, '_31_60_days': 4, '_61_90_days': 5, '_91_120_days': 6, '_greater_than_120_days': 7}),
 Row((None, 'EXPRESS', 'Mid Atlantic', 2934, 2982, 2982, 0, 0), {'f0_': 0, 'sm_type': 1, 'cc_name': 2, '_30_days': 3, '_31_60_days': 4, '_61_90_days': 5, '_91_120_days': 6, '_greater_than_120_days': 7}),
 Row((None, 'EXPRESS', 'NY Metro', 2848, 2864, 3110, 0, 0), {'f0_': 0, 'sm_type': 1, 'cc_name': 2, '_30_days': 3, '_31_60_days': 4, '_61_90_days': 5, '_91_120_days': 6, '_greater_than_120_days': 7}),
 Row((None, 'EXPRESS', 'North Midwest', 3000, 2962, 2932, 0, 0), {'f0_': 0, 'sm_type': 1, 'cc_name': 2, '_30_days': 3, '_31_60_days': 4, '_61_90_days': 5, '_91_120_days': 6, '_greater_than_120_days': 7}),
 Row((None, 'EXPRESS', 'Pacific Northwest', 3018, 3082, 2998, 0, 0), {'f0_': 0, 'sm_type': 1, 'cc_name': 2, '_30_days': 3, '_31_60_days': 4, '_61_90_days': 5, '_91_120_days': 6, '_greater_t

In [177]:
break

SyntaxError: 'break' outside loop (<ipython-input-177-6aaf1f276005>, line 1)